# Pijpleidingen

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Lees de gegevens
data_bastand = "../data/data.csv"
data = pd.read_csv(data_bastand)

# Scheid doel van voorspellers
y = data.Price
X = data.drop(['Price'], axis=1)

# Verdeel gegevens in trainings- en validatiesubsets
X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Numerieke kolommen selecteren
numerieke_kolommen = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# "Kardinaliteit" betekent het aantal unieke waarden in een kolom
# Selecteer categorische kolommen met relatief lage kardinaliteit (handig maar willekeurig)
categorisch_kolommen = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

# Alleen geselecteerde kolommen behouden
mijn_kolommen = categorisch_kolommen + numerieke_kolommen
X_train = X_train_full[mijn_kolommen].copy()
X_test = X_test_full[mijn_kolommen].copy()

In [ ]:
X_train.head()

## Stap 1: Definieer preprocessingstappen

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing voor numerieke gegevens
numeriche_transformator = SimpleImputer(strategy='constant')

# Preprocessing voor categorische gegevens
categorisch_transformator = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

# Bundle preprocessing voor numerieke en categorische gegevens
preprocessor = ColumnTransformer(
transformers=[
    ('num', numeriche_transformator, numerieke_kolommen),
    ('cat', categorisch_transformator, categorisch_kolommen)
    ])

## Stap 2: Definieer het model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

## Stap 3: De pijplijn maken en evalueren

In [ ]:
from sklearn.metrics import mean_absolute_error

# Bundel preprocessing en modellering van code in een pijplijn
mijn_pijpleiding = Pipeline(steps=[('preprocessor', preprocessor),('model', model)])

# Preprocessing van trainingsgegevens, fit model
mijn_pijpleiding.fit(X_train, y_train)

# Preprocessing van validatiegegevens, voorspellingen ophalen
preds = mijn_pijpleiding.predict(X_test)

# Evalueer het model
score = mean_absolute_error(y_test, preds)
print('MAE:', score)

In [ ]:
# Preprocessing van testgegevens, fit model
preds_test= mijn_pijpleiding.predict(X_test)
preds_test

In [ ]:
# Testvoorspellingen opslaan in bestand
output = pd.DataFrame({'Id': X_test.index,'SalePrice': preds_test})
output.to_csv('uitkomst.csv', index=False)